# Monitoring functional metrics 🔍

You may have noticed, built-in operational metrics don't tell us everything about our system.

In this notebook, we will therefore add custom log-based metrics to our application using Azure Application Insights, and then use them to monitor the system.

## Sending structured logs to cloud

You may know regular logging. Structured logging is not that different, only we can log a JSON object instead of a string. This makes it easier to parse and analyze logs later on as we do not need to extract information from the standard string-based logs.

In [1]:
import logging
import structlog
from structlog.processors import JSONRenderer

logging.basicConfig(level=logging.INFO)
standard_logger = logging.getLogger()

structlog.configure(processors=[JSONRenderer()])
structlog_logger = structlog.get_logger()

In [2]:
standard_logger.info("This is a standard log!")

In [3]:
structlog_logger.info("This is a structured log!", accuracy=0.99)

{"accuracy": 0.99, "event": "This is a structured log!"}


In [4]:
from llmops_training.news_reader.logs import configure_structlog
import structlog

In [5]:
configure_structlog()

In [6]:
logger = structlog.get_logger()


In [ ]:
logger.info({
    "event": "structured logging initialized",
    "status": "success"
})

INFO:IPython.core.oinspect:{"event": {"event": "structured logging initialized", "status": "success"}, "level": "info", "timestamp": "2025-12-17T09:32:28.882244Z"}


INFO:azure.core.pipeline.policies.http_logging_policy:Request URL: 'https://westeurope-5.in.applicationinsights.azure.com//v2.1/track'
Request method: 'POST'
Request headers:
    'Content-Type': 'application/json'
    'Content-Length': '4883'
    'Accept': 'application/json'
    'x-ms-client-request-id': '4f97172c-db2b-11f0-ac66-3e10b6f74135'
    'User-Agent': 'azsdk-python-azuremonitorclient/unknown Python/3.12.9 (macOS-26.2-arm64-arm-64bit)'
A body is sent with the request
INFO:azure.core.pipeline.policies.http_logging_policy:Response status: 200
Response headers:
    'Transfer-Encoding': 'chunked'
    'Content-Type': 'application/json; charset=utf-8'
    'Server': 'Microsoft-HTTPAPI/2.0'
    'Strict-Transport-Security': 'REDACTED'
    'X-Content-Type-Options': 'REDACTED'
    'Date': 'Wed, 17 Dec 2025 09:32:33 GMT'
INFO:azure.monitor.opentelemetry.exporter.export._base:Transmission succeeded: Item received: 4. Items accepted: 4
INFO:azure.core.pipeline.policies.http_logging_policy:Re

As you see, with structlog we can add arbitary key-value pairs that appear in the structured log. <br>
This is particularly useful for adding custom metrics for monitoring.

Now, what custom metrics should we monitor for our application? 🤔

Let's start by keeping track of the number articles uploaded to the application.
This will help us monitor whether users actually make use of the application, which is always good to know.

> **Exercise** 📝
>
> - Inspect the `app.py` file to see how we have configured the `structlog` logger. Most of the configuration is necessary for the logger to send logs to the Azure Logging service.
> - Now add a log statement in `components.py` to log the number of articles added to the application. You can find the TODO at `# TODO(11-monitor-functional-metrics)`.
> - Run the application and upload some articles to trigger the log statement.
>
> 💡 Hints:
>
> - You can use the `logger` object that's already available in the `components.py` file.
> - You can add a custom event, e.g. `articles_added`, with a custom key, e.g. `number_articles_added`, with a corresponding value to the log statement.
> - To make subsequent steps easier, i.e. querying the logs in Azure, you can add a personal identifier key to the log statement, e.g. `user_name`, with your user name.
> - You can use `make proxy` to access the deployed application. But first, you will need to deploy a new revision. You can do this using: `make build-push-deploy`.
> 

## Querying logs in explorer and adding custom metric

Now the logs should be available in Azure Application Insights.
Before we can visualize our custom metric, we can query the logs using Kusto Query Language (KQL).

Let's see if we can query our logs, and then create visualizations from them.

> **Exercise** 📝
>
> - Navigate to Azure Application Insights in the Azure portal and find the Logs section.
> - In the query field, add a KQL query to filter your logs. An example query can be:
>   ```kql
>   traces
>   | where customDimensions.event == "articles_added"
>   ```
> - Do you see your own logs? Now you can create custom metrics and charts from these logs.
> - In Azure Monitor, navigate to Metrics and select your Application Insights resource
> - Create a custom metric based on your log query
> - Give it a descriptive name so you can find it back, e.g. `{USER_NAME}-number-articles-added`
> - Now, trigger the log line in your application by extracting some articles and check if the metric becomes available in the metrics explorer.
> - Create a widget in your dashboard that visualizes this metric over time.
>
> 💡 Hints:
>
> - If you can't filter your logs successfully, try to query for all traces first. Then inspect how the logs look like and add filter statements to your query one by one.
> - All the structured log keys are available in the `customDimensions` field. The `customDimensions.event` key is the "message" of the log statement.
> - You can also use Azure Workbooks to create more complex visualizations and dashboards.

## (Bonus) Extend dashboard with more custom metrics

> **(Bonus) Exercise** 📝 
>
> - Add more custom log-based metrics to the application, and monitor them in the dashboard.
> - Think about:
>   - The number of tokens going to and from the LLM
>   - The number of request to the LLM
>   - Success rates of the requests
>   - LLM response latency
>   - How often the LLM output is saved to the data base
>   - (Later) Number of positive user feedback
>   - (Later) Number of negative user feedback